# Text Analysis on putinmissing data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,576171827086520320,1093610112,#PUTIN FULL OF LEAD\n #ПутинУмер #putindead ht...,Fri Mar 13 00:04:02 +0000 2015,0
1,576185920094908416,458173,Surprised it took me so long to stumble across...,Fri Mar 13 01:00:03 +0000 2015,0
2,576249853979807745,44691136,#PutinDead - Amazing how quickly the satirical...,Fri Mar 13 05:14:06 +0000 2015,0
3,576316647880839168,285312699,#putindead I heard it from a good source that ...,Fri Mar 13 09:39:30 +0000 2015,0
5,576321776424189954,8628872,Enter 'Putin' into Twitter search. Look at all...,Fri Mar 13 09:59:53 +0000 2015,0
...,...,...,...,...,...
683,577294310573502465,15300388,"Israel’s Ex-Ambassador To Russia: ""There Are S...",Mon Mar 16 02:24:23 +0000 2015,1
691,577297951325876224,82113500,Why Putin’s “Coup” Might be a Staged Event to ...,Mon Mar 16 02:38:51 +0000 2015,1
692,577301342617673729,15300388,Israel’s Ex-Ambassador To Russia: “There Are S...,Mon Mar 16 02:52:20 +0000 2015,1
693,577303162765250560,260076391,Israel’s Ex-Ambassador To Russia: “There Are S...,Mon Mar 16 02:59:34 +0000 2015,1


## Prepocessing
- Define function to process text at one time

In [4]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
        
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [5]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  
cleaned text:  


### applied on original data (**df**)

In [6]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [7]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,576171827086520320,1093610112,#PUTIN FULL OF LEAD\n #ПутинУмер #putindead ht...,Fri Mar 13 00:04:02 +0000 2015,5.763882e+17,1.317093e+09,"@CWynnykWilson ""Gas up!""",Fri Mar 13 14:23:53 +0000 2015,0,putin full lead путинумер putindead,gas
1,576185920094908416,458173,Surprised it took me so long to stumble across...,Fri Mar 13 01:00:03 +0000 2015,NaN,NaN,,NaN,0,surprised took long stumble across image today...,
2,576249853979807745,44691136,#PutinDead - Amazing how quickly the satirical...,Fri Mar 13 05:14:06 +0000 2015,NaN,NaN,,NaN,0,putindead amazing quickly satirical songs vide...,
3,576316647880839168,285312699,#putindead I heard it from a good source that ...,Fri Mar 13 09:39:30 +0000 2015,5.763988e+17,3.077746e+09,@nikkimcwatters http://t.co/xSFvkAnTje,Fri Mar 13 15:05:48 +0000 2015,0,putindead heard good source vladimir putin rid...,
4,576316647880839168,285312699,#putindead I heard it from a good source that ...,Fri Mar 13 09:39:30 +0000 2015,5.794791e+17,8.445691e+07,@nikkimcwatters \nI wish I understood why Puti...,Sun Mar 22 03:06:09 +0000 2015,0,putindead heard good source vladimir putin rid...,wish understood putin despised hes got better ...


### applied on source tweet data (**src_tw_df**) 

In [8]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [11]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [12]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,576171827086520320,1093610112,#PUTIN FULL OF LEAD\n #ПутинУмер #putindead ht...,Fri Mar 13 00:04:02 +0000 2015,5.763882e+17,1.317093e+09,"@CWynnykWilson ""Gas up!""",Fri Mar 13 14:23:53 +0000 2015,0,putin full lead путинумер putindead,gas,Neutral,Neutral


### applied on source tweet data only 

In [13]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/putinmissing/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/putinmissing/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/putinmissing/senti_source.pdf',dpi=300)

### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/putinmissing/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/putinmissing/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/putinmissing/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [14]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [16]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['emotion']=emo

In [17]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-df.csv',index=False)

In [18]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['emotion']=emo1

In [19]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-src.csv',index=False)